In [1]:
#AO star
class Node:
    def __init__(self, name, heuristic=0):
        self.name = name
        self.heuristic = heuristic
        self.children = []  # List of tuples (child, cost)
        self.type = 'OR'  # Node type can be 'AND' or 'OR'
        self.parent = None
        self.cost = 0  # Cost to reach this node from its parent

    def add_child(self, child, cost):
        self.children.append((child, cost))
        child.parent = self


def ao_star(start_node):
    open_list = set()
    closed_list = set()

    def recursive_ao_star(node):
        if node.children == []:
            return node.heuristic

        if node.type == 'OR':
            min_cost = float('inf')
            best_child = None
            for child, cost in node.children:
                cost_to_goal = cost + recursive_ao_star(child)
                if cost_to_goal < min_cost:
                    min_cost = cost_to_goal
                    best_child = child
            node.cost = min_cost
            return node.cost

        elif node.type == 'AND':
            total_cost = 0
            for child, cost in node.children:
                total_cost += cost + recursive_ao_star(child)
            node.cost = total_cost
            return node.cost

    solution_path = []

    def find_solution_path(node):
        if node:
            solution_path.append(node.name)
            if node.children:
                if node.type == 'OR':
                    best_child = min(node.children, key=lambda x: x[1] + x[0].cost)[0]
                    find_solution_path(best_child)
                elif node.type == 'AND':
                    for child, _ in node.children:
                        find_solution_path(child)

    recursive_ao_star(start_node)
    find_solution_path(start_node)

    return solution_path


# Example graph construction
# Define nodes
S = Node('S', 5)
A = Node('A', 3)
B = Node('B', 4)
C = Node('C', 2)
D = Node('D', 6)
G = Node('G', 0)

# Set node types
S.type = 'OR'
A.type = 'OR'
B.type = 'AND'
C.type = 'OR'
D.type = 'OR'

# Define graph relationships
S.add_child(A, 1)
S.add_child(G, 10)
A.add_child(B, 2)
A.add_child(C, 1)
B.add_child(D, 5)
C.add_child(D, 3)
C.add_child(G, 4)
D.add_child(G, 2)

# Perform AO* search
solution = ao_star(S)

print("Best path found by AO* algorithm:")
print(" -> ".join(solution))


Best path found by AO* algorithm:
S -> A -> C -> G


In [2]:
class Node:
    def __init__(self, name, is_and_node=False, heuristic=0.0):
        self.name = name
        self.is_and_node = is_and_node
        self.heuristic = heuristic
        self.cost = float('inf')
        self.solved = False
        self.children = []  # List of (child_node, cost)
        self.best_path = []  # To store the best path from this node

    def add_child(self, child, cost):
        self.children.append((child, cost))


def ao_star(node, path):
    if node.solved:
        path.extend(node.best_path)
        return node.cost

    if not node.children:
        node.solved = True
        node.cost = node.heuristic
        node.best_path.append(node)
        path.append(node)
        return node.cost

    if node.is_and_node:
        node.cost = 0.0
        node.best_path.clear()  # Clear any previous paths
        for child, cost in node.children:
            child_path = []
            node.cost += cost + ao_star(child, child_path)
            node.best_path.append(child)  # Add the AND node's child
            node.best_path.extend(child_path)

    else:  # OR node
        min_cost = float('inf')
        best_child_path = []

        for child, cost in node.children:
            child_path = []
            total_cost = cost + ao_star(child, child_path)
            if total_cost < min_cost:
                min_cost = total_cost
                best_child_path = child_path
                node.best_path.clear()  # Clear any previous paths
                node.best_path.append(child)  # Add the OR node's best child

        node.cost = min_cost
        node.best_path.extend(best_child_path)

    node.solved = True
    path.extend(node.best_path)
    return node.cost


def print_path(path):
    visited = set()
    for node in path:
        if node.name not in visited:
            print(node.name, end=" ")
            visited.add(node.name)
    print()


if __name__ == "__main__":
    root = Node("Start")

    a = Node("A", is_and_node=False, heuristic=0.0)
    b = Node("B", is_and_node=True, heuristic=6.0)
    c = Node("C", is_and_node=False, heuristic=12.0)
    d = Node("D", is_and_node=True, heuristic=10.0)
    e = Node("E", is_and_node=False, heuristic=4.0)
    f = Node("F", is_and_node=False, heuristic=4.0)
    g = Node("G", is_and_node=False, heuristic=5.0)
    h = Node("H", is_and_node=False, heuristic=7.0)

    root.add_child(a, 0.0)
    a.add_child(b, 1.0)
    a.add_child(c, 1.0)
    a.add_child(d, 1.0)
    b.add_child(g, 1.0)
    b.add_child(h, 1.0)
    d.add_child(e, 1.0)
    d.add_child(f, 1.0)

    path = []
    result = ao_star(root, path)
    print(f"Minimum cost: {result}")
    print("Path: ", end="")
    print_path(path)


Minimum cost: 11.0
Path: A D E F 


In [7]:
def Cost(H, condition, weight=1):
    cost = {}
    if 'AND' in condition:
        AND_nodes = condition['AND']
        Path_A = ' AND '.join(AND_nodes)
        PathA = sum(H[node] + weight for node in AND_nodes)
        cost[Path_A] = PathA
    if 'OR' in condition:
        OR_nodes = condition['OR']
        Path_B = ' OR '.join(OR_nodes)
        PathB = min(H[node] + weight for node in OR_nodes)
        cost[Path_B] = PathB
    return cost
def update_cost(H, Conditions, weight=1):
    Main_nodes = list(Conditions.keys())
    Main_nodes.reverse()
    least_cost = {}
    for key in Main_nodes:
        condition = Conditions[key]
        c = Cost(H, condition, weight)
        H[key] = min(c.values())
        least_cost[key] = Cost(H, condition, weight)
    return least_cost
def shortest_path(Start, Updated_cost, H):
    Path = Start
    total_cost = H[Start] if Start not in Updated_cost else min(Updated_cost[Start].values())
    if Start in Updated_cost.keys():
        Min_cost = min(Updated_cost[Start].values())
        key = list(Updated_cost[Start].keys())
        values = list(Updated_cost[Start].values())
        Index = values.index(Min_cost)
        Next = key[Index].split()
        if len(Next) == 1:
            Start = Next[0]
            sub_path, _ = shortest_path(Start, Updated_cost, H)
            Path += '-' + sub_path
        else:
            Path += '-(' + key[Index] + ') '
            Start = Next[0]
            sub_path_1, _ = shortest_path(Start, Updated_cost, H)
            Start = Next[-1]
            sub_path_2, _ = shortest_path(Start, Updated_cost, H)
            Path += '[' + sub_path_1 + ' + ' + sub_path_2 + ']'
    return Path, total_cost
H = {'A': -1, 'B': 6, 'C': 12, 'D': 10, 'E': 4, 'F': 4, 'G': 5, 'H': 7}
Conditions = {
    'A': {'OR': ['D'], 'AND': ['B', 'C']},
    'B': {'OR': ['G', 'H']},
    'D': {'AND': ['E', 'F']}
}
weight = 1
Updated_cost = update_cost(H, Conditions, weight)
path, least_cost = shortest_path('A', Updated_cost, H)
print('Shortest Path:\n', path)
print(f'Least Cost at A: {least_cost}')


Shortest Path:
 A-D-(E AND F) [E + F]
Least Cost at A: 11
